In [31]:

import pandas as pd
import numpy as np
import os

import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import altair as alt
from branca.colormap import LinearColormap
from vega_datasets import data
import folium
from streamlit_folium import folium_static, st_folium

import pmdarima as pm
from pmdarima.pipeline import Pipeline
from pmdarima.preprocessing import BoxCoxEndogTransformer
from pmdarima.preprocessing import LogEndogTransformer
from sklearn.metrics import r2_score, mean_absolute_error, mean_absolute_percentage_error, root_mean_squared_error, mean_squared_error

import tensorflow as tf
from tensorflow.keras.saving import load_model
from joblib import dump, load
from sklearn.preprocessing import MinMaxScaler

import random

import warnings
warnings.filterwarnings('ignore')

root = '.'
#root = 'streamlit'

# Liste des zones d'emploi
df_zones = pd.read_csv(f'{root}/data/processed/dvm/dvm_par_zone/zone_name.csv')
path_dvm = f'{root}/data/processed/dvm/dvm_par_zone'
path_models_dvm = f'{root}/data/processed/dvm/models'

# Récupération map
geo_zone_emploi = gpd.read_file('./data/referentiels/geojson/zone_emploi_idf_normandie.geojson')
geo_zone_emploi = geo_zone_emploi.set_index('libze2020')

# Récupération map nombre indicateur durée de vie moyenne entre 2011-2015 et 2016-2020
geo_dvm_2020 = gpd.read_file(f'{path_dvm}/geo_dvm_communes_idf_normandie_2020.geojson')
geo_dvm_2020 = geo_dvm_2020.set_index('nom')

# renvoi le dataframe de la durée de vie moyenne d'une zone d'emploi
def get_dvm_per_zone(zone_emploi) :
    df_zone = pd.read_csv(f'{path_dvm}/dvm_{zone_emploi}.csv')
    df_zone = df_zone.ffill()
    return df_zone


# Renvoi une liste de toutes les zones d'emploi
def get_list_zone_emploi():
    return df_zones['zone_name'].value_counts().reset_index()['zone_name'].to_list()

# Renvoi un pointeur d'affichage des durées de vie pour les zones d'emploi selectionnées
def display_dvm(selected) :

    list_dvm = []
    for zone_emploi in selected :
        df = pd.read_csv(f'{path_dvm}/dvm_{zone_emploi}.csv')
        df = df.rename(columns={"duree_observation_etab": zone_emploi, "dateDebut" : "Années"})

        if 'df_final' in locals():
            df_final = df_final.merge(df, on='Années', how='inner')
        else :
            df_final = df.copy()
    
    fig = px.line(df_final, x='Années', y=selected)
    
    return fig

# Affiche une carte des communes et des données d'évolution de la durée de vie moyenne
def display_map() :

    fig = px.choropleth(geo_zone_emploi,
                   geojson=geo_zone_emploi.geometry,
                   locations=geo_zone_emploi.index,
                   color="nb_com",
                   color_continuous_scale="Viridis",
                   projection="mercator")
    
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.update_geos(fitbounds="locations", visible=False)

    return fig

# Affiche les données associées au ratio la durée de vie moyenne entre 2011-2015 et 2016-2020
def display_dvm_2020(pow) :    

    fig = px.choropleth(geo_dvm_2020,
                   geojson=geo_dvm_2020.geometry,
                   locations=geo_dvm_2020.index,
                   color="indicateur_dvm",
                   range_color=[0.5,1.5],
                   color_continuous_scale="viridis",
                   projection="mercator")
    
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.update_geos(fitbounds="locations", visible=False)

    return fig

2024-09-15 20:43:49.779 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [5]:
# Affiche les prédictions
def display_lstm_bi_prediction(zone_emploi, n_forecast, sequence_type='lowseq') :

    # Récupération du modèle
    loaded_model = load_model(f'{path_models_dvm}/rnn_lstm_bi_{sequence_type}_{zone_emploi}.keras')
    df_dvm = get_dvm_per_zone(zone_emploi)

    if sequence_type == 'lowseq' :
        sequence_length = 5
    else :
        sequence_length = 12 

    __, X_test, __, __, __ = build_scaled_train_test(df_dvm['duree_observation_etab'], n_forecast, sequence_length)

    X_test = X_test[:n_forecast]
    y_pred = loaded_model.predict(X_test)
    fig = display_prediction_over_test(df_dvm, y_pred, n_forecast)

    return fig

# Affiche les prédictions sarima log
def display_sarima_log(zone_emploi, n_forecast, type) :

    # Récupération du modèle
    loaded_model = load(f'{path_models_dvm}/sarima_{type}_{zone_emploi}')
    df_dvm = get_dvm_per_zone(zone_emploi)
    
    y_pred = loaded_model.predict(n_periods=n_forecast).values
    fig = display_prediction_over_test(df_dvm, y_pred, n_forecast)

    return fig

# Transformation des données et split ( normalisation )
def build_scaled_train_test(serie, forecast, LOOK_BACK) :

    X1, y1 = df_to_X_y(serie, LOOK_BACK)
    shift = (len(serie) - forecast) - LOOK_BACK

    X_train, y_train = X1[:shift], y1[:shift]
    X_test, y_test = X1[shift:], y1[shift:]

    # Reshape des tableaux avant la normalisation 
    X_train = X_train.reshape(X_train.shape[0], LOOK_BACK)
    X_test = X_test.reshape(X_test.shape[0], LOOK_BACK)

    # Normalisation des données 
    scaler = MinMaxScaler(feature_range=(-1, 1))
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Reshape des données normalisées en matrice
    X_train = X_train_scaled.reshape(X_train.shape[0], LOOK_BACK, 1)
    X_test = X_test_scaled.reshape(X_test.shape[0], LOOK_BACK, 1)

    return X_train, X_test, y_train, y_test, scaler

# Transformation en données supervisées ( lag )
def df_to_X_y(df, window_size=5):
  df_as_np = df.to_numpy()
  X = []
  y = []
  for i in range(len(df_as_np)-window_size):
    row = [[a] for a in df_as_np[i:i+window_size]]
    X.append(row)
    label = df_as_np[i+window_size]
    y.append(label)
  return np.array(X), np.array(y)

def display_prediction_over_test(df_duree_moyenne, y_pred, n_forecast) :

    df_past = df_duree_moyenne[0:-60]
    df_past = df_past.rename(columns={"duree_observation_etab": "values"})    
    y_test = np.array(df_duree_moyenne[df_past.shape[0]:(df_past.shape[0]+n_forecast)]['duree_observation_etab'])

    df_future = pd.DataFrame(columns=['dateDebut','values'])
    df_future['dateDebut'] = pd.date_range(start=df_past['dateDebut'].iloc[-1], freq ='M', periods=n_forecast+1)
    df_future.drop(0, inplace=True)
    df_future['dateDebut'] = pd.to_datetime(df_future['dateDebut'])
    df_future['dateDebut'] = df_future['dateDebut'].dt.strftime('%Y-%m')
    df_future['values'] = y_pred

    df_test = pd.DataFrame(columns=['dateDebut','values'])
    df_test['dateDebut'] = pd.date_range(start=df_past['dateDebut'].iloc[-1], freq ='M', periods=n_forecast+1)
    df_test.drop(0, inplace=True)
    df_test['dateDebut'] = pd.to_datetime(df_future['dateDebut'])
    df_test['dateDebut'] = df_test['dateDebut'].dt.strftime('%Y-%m')
    df_test['values'] = y_test

    trace = go.Scatter(
        x = df_past['dateDebut'],
        mode='lines',
        y=df_past['values'],
        marker=dict(color='rgba(12, 124, 32, 0.5)'),
        name='duree moyenne'
        )  
        
    trace1 = go.Scatter(
            x = df_future['dateDebut'],
            mode='lines',
            y=df_future['values'],
            marker=dict(color='blue'),
            name='duree moyenne predite'
        )

    trace2 = go.Scatter(
            x = df_test['dateDebut'],
            mode='lines',
            y=df_test['values'],
            marker=dict(color='red'),
            name='duree moyenne réelle'
        )

    layout = dict(title=f'Forecast sur {n_forecast} mois - comparaison avec données de test')
    data = [trace, trace1, trace2]
    fig = dict(data=data, layout=layout)

    return fig


def calculate_metrique() :
        
    df_metriques_dvm = pd.DataFrame(columns = ['ze', 'sarima_mape', 'sarima_rmse', 'sarima_ratio', 'lstm_bi_mape', 'lstm_bi_rmse', 'lstm_bi_ratio'])
      
    # pour chaque zone
    for zone in get_list_zone_emploi() :
        df_file_zone = get_dvm_per_zone(zone)
        df_file_zone = df_file_zone.ffill()

        # sarima
        loaded_model_sarima = load(f'{path_models_dvm}/sarima_prediction_log_{zone_emploi}')
        
        y_pred = loaded_model_sarima.predict(n_periods=60).values
        df_past = df_file_zone[0:-60]
        y_test = np.array(df_file_zone[df_past.shape[0]:(df_past.shape[0]+60)]['duree_observation_etab'])

        r2_value_sarima = r2_score(y_test, y_pred)
        mape_value_sarima = mean_absolute_percentage_error(y_test, y_pred)
        rmse_value_sarima = root_mean_squared_error(y_test, y_pred)
        ratio_value_sarima = (y_pred / y_test).mean()

        
        #LSTM bidirectionnel    
        loaded_model_lstm_bi = load_model(f'{path_models_dvm}/rnn_lstm_bi_highseq_{zone}.keras')

        __, X_test, __, __, __ = build_scaled_train_test(df_file_zone['duree_observation_etab'], 60, 12)

        X_test = X_test[:60]
        y_pred = loaded_model_lstm_bi.predict(X_test)
        
        r2_value_lstm = r2_score(y_test, y_pred)
        mape_value_lstm = mean_absolute_percentage_error(y_test, y_pred)
        rmse_value_lstm = root_mean_squared_error(y_test, y_pred)
        ratio_value_lstm = (y_pred / y_test).mean()

        df_metriques_dvm = df_metriques_dvm._append({'ze': zone, 'sarima_mape': mape_value_sarima, 'sarima_rmse': rmse_value_sarima, 'sarima_ratio' : ratio_value_sarima,
                                                                'lstm_bi_mape' : mape_value_lstm, 'lstm_bi_rmse' : rmse_value_lstm, 'lstm_bi_ratio' : ratio_value_lstm}, ignore_index=True)
        

    return df_metriques_dvm

    

In [6]:
def display_metrics_dvm() :
    df_metrics_dvm = pd.read_csv(f'{path_dvm}/metrics_evaluation_lstm_sarima.csv')

    plt.figure(figsize=(15,6))
    df_melted = pd.melt(df_metrics_dvm, ['ze'])
    fig = px.line(data_frame=df_melted, x='ze', y='value', color='variable')
    
    return fig

display_metrics_dvm()

<Figure size 1500x600 with 0 Axes>

In [59]:
#test.to_csv('../streamlit/data/processed/dvm/dvm_par_zone/metrics_evaluation_lstm_sarima.csv', encoding='utf-8', index=False)

In [10]:
geo_zone_emploi.head()

,ze2020,nb_com,geometry
libze2020,,,
Caen,2804,380,"POLYGON ((-0.36971 48.83553, -0.36889 48.84181..."
Rouen,2815,300,"POLYGON ((1.08333 49.20886, 1.08124 49.19948, ..."
Dieppe-Caux maritime,2807,196,"POLYGON ((1.59977 49.61966, 1.60356 49.6328, 1..."
Evreux,2808,167,"POLYGON ((1.06303 48.75897, 1.03521 48.7399, 1..."
Cherbourg en Cotentin,2805,135,"MULTIPOLYGON (((-1.47882 49.36495, -1.49757 49..."


In [44]:
def build_map_zone_emploi(field):   

    df = geo_zone_emploi

    if field=="nb_com":
        # Custom diverging color map (Green at 1, red below and above)
        custom_colormap = LinearColormap(
            colors=['red', 'orange', 'green','orange', 'red'],  # Red for <1, Green at 1, Yellow for >1
            #vmin=df['ratio'].min(),  # Minimum value of your 'ratio' column
            vmin=50,
            vmax=400,
            caption='Ratio (vert = meilleur)'
        ).scale(50, 400)
       
        
    # Create a folium map centered around your geometries
    m = folium.Map(
        location=[48.91, 0.71], 
        zoom_start=8, prefer_canvas=True)

    # Function to style each feature (polygon) based on 'ratio' value
    def style_function(feature):
        field_value = feature['properties'][field]
        return {
            'fillColor': custom_colormap(field_value),  # Get color from colormap
            'color': 'white',
            'weight': 0.5,
            'fillOpacity': 0.7,
        }

    # Add the GeoJson layer to the map, including the ratio for styling
    folium.GeoJson(
        df,
        style_function=style_function,
        #tooltip=folium.GeoJsonTooltip(fields=[df.index, field])  # Display additional fields in tooltips
    ).add_to(m)

    # Get the bounds (bounding box) of the GeoDataFrame to fit the map to the geometry
    bounds = df.total_bounds  # [minx, miny, maxx, maxy]

    # Fit the map to the bounds of the geometry (bbox)
    m.fit_bounds([[bounds[1], bounds[0]], [bounds[3], bounds[2]]])

    # Add the custom colormap to the map as a legend
    custom_colormap.add_to(m)

    return m

In [41]:
m = build_map_zone_emploi('nb_com')   
m.save("footprint.html")

,ze2020,nb_com,geometry
libze2020,,,
Caen,2804,380,"POLYGON ((-0.36971 48.83553, -0.36889 48.84181..."
Rouen,2815,300,"POLYGON ((1.08333 49.20886, 1.08124 49.19948, ..."
Dieppe-Caux maritime,2807,196,"POLYGON ((1.59977 49.61966, 1.60356 49.6328, 1..."
Evreux,2808,167,"POLYGON ((1.06303 48.75897, 1.03521 48.7399, 1..."
Cherbourg en Cotentin,2805,135,"MULTIPOLYGON (((-1.47882 49.36495, -1.49757 49..."


In [13]:
df_metrics_dvm = pd.read_csv(f'{path_dvm}/metrics_evaluation_lstm_sarima.csv')
df_metrics_dvm.head()

,ze,sarima_mape,sarima_rmse,sarima_ratio,lstm_bi_mape,lstm_bi_rmse,lstm_bi_ratio
0,Rouen,0.102012,0.911455,1.065311,0.080070,0.765670,0.996410
1,Evry,0.261523,1.709426,1.227953,0.144739,1.451585,1.011895
2,Dreux,0.304497,2.461677,1.085874,0.460884,3.820606,1.176418
3,Beauvais,0.409314,4.704115,0.888497,0.530634,4.705886,1.153210
4,Paris,0.346383,1.872122,1.330149,0.106444,0.873496,0.981360


In [16]:

import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Box(y=df_metrics_dvm['sarima_ratio'],  name="ratio moyen sarima"))
fig.add_trace(go.Box(y=df_metrics_dvm['lstm_bi_ratio'], name="ratio moyen LSTM"))
fig.update_traces(boxpoints='all', jitter=0.3)
fig.show()

